### This project requires Python 3.7 or above:

In [1]:
import sys

assert sys.version_info >= (3, 7), "This script requires Python 3.7 or higher!"

### It also requires Scikit-Learn ≥ 1.0.1:

In [2]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pathlib import Path # Use pathlib.Path for cleaner, object-oriented, and cross-platform file path handling

### MNIST dataset

In [ ]:
# load the MNIST dataset and check the keys of the mnist object
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
mnist.keys()

# split the data into features and target values
X = mnist.iloc[:, :-1]  # Features
y = mnist.iloc[:, -1]   # Target

### Fashion mnist

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
 # Load dataset
(x_train_full, y_train_full), (x_test_full, y_test_full) = fashion_mnist.load_data()

### Take a Quick Look at the Data Structure

In [ ]:
data.head()                                  #look at the top five rows of data
data.info()                                  #get a quick description of the data
data.describe()                              #shows a summary of the numerical attributes
data["column_name"].value_counts()           #for categorical attribute (object)

In [ ]:
import matplotlib.pyplot as plt 
housing.hist(bins=50, figsize=(12, 8))       #call the hist() method on the whole dataset and it will plot a histogram 
plt.show()                                   #for each numericaattribute

### Create a Test Set 80% and 20%

In [ ]:
from sklearn.model_selection import train_test_split

#most common when you already have your X (features) and y (labels) split
train_X, test_X, train_y, test_y = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,                              #ensures class balance in the train/test sets
    random_state=42
)

#----------------------------------------------------------or------------------------------------------------------------------

train_set, test_set = train_test_split(
    data,
    test_size=0.2,                           #Split the dataset into a training and a testing set retaining 80% and 20%
    stratify=data["target_column"],                              
    random_state=42
)


# Split the dataset into features (X) and target (y)
# X includes every column except the last one (quality) and y includes only the last
X_train = train_set.iloc[:, :-1]  
y_train = train_set.iloc[:, -1] 
X_test = test_set.iloc[:, :-1]    
y_test = test_set.iloc[:, -1] 

### training (5/7), a validation (1/7), and a test (1/7) set

In [ ]:
# Load MNIST dataset
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X, y = mnist["data"], mnist["target"].astype(np.uint8)

# Step 1: split into 5/7 train and 2/7 temp (validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=2/7, random_state=42, stratify=y)

# Step 2: split temp into 1/7 val and 1/7 test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Confirm shapes
print("Train set:", X_train.shape, y_train.shape)
print("Validation set:", X_val.shape, y_val.shape)
print("Test set:", X_test.shape, y_test.shape)

### Clean the Data

In [ ]:
#Most machine learning algorithms cannot work with missing features, so you’ll need to take care of these
 #1. Get rid of the corresponding districts.
 #2. Get rid of the whole attribute.
 #3. Set the missing values to some value (zero, the mean, the median,etc.). This is called imputation.
#You can accomplish these easily using the Pandas DataFrame’s dropna(), drop(), and fillna() methods:

data.dropna(subset=["column_name"], inplace=True)     # option 1 Delete all the rows with NaN
data.drop("column_name", axis=1)                      # option 2 Delete the entire column
median = data["column_name"].median()                 # option 3 Replace all NaN with the median or sklearn.SimpleImputer
data["column_name"].fillna(median, inplace=True)

#SimpleImpute: The benefit is that it will store the median value of each feature.
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
data_num = data.select_dtypes(include=[np.number])    #median can only be computed on numerical attributes
imputer.fit(data_num)
imputer.statistics_
data_num.median().values
X = imputer.transform(data_num)
#( There are also more powerful imputers available in the sklearn.impute package)

### Feature Scaling and Transformation

In [ ]:
#One of the most important transformations you need to apply to your data is feature scaling.
#There are two common ways to get all attributes to have the same scale:min-max scaling(normalization) and standardization

from sklearn.preprocessing import StandardScaler            
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)           #use fit() or fit_transform() to the training data only
X_test_scaled = scaler.transform(X_test)                 #use transform() to the validation set, the test set, and new data


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)          #Fit only to training set
X_valid_scaled = scaler.transform(X_valid)              #Transform
X_test_scaled  = scaler.transform(X_test)               #Transform


### LinearRegression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

scaler = StandardScaler()                              # Scale the input features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lin_reg_model = LinearRegression()                     # Create the Linear Regression model

lin_reg_model.fit(X_train_scaled, y_train)             # Fit the model to training data

y_pred_lin = lin_reg_model.predict(X_test_scaled)      # Predict on the test set

print("Linear Regression:")                            # Evaluate the model
print(f"RMSE: {mean_squared_error(y_test, y_pred_lin, squared=False):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred_lin):.2f}")

### SVM classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

scaler = StandardScaler()                                 # Scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm_model = SVC(kernel='linear', C=0.1, random_state=42)  # Create the SVM model

svm_model.fit(X_train_scaled, y_train)                    # Train the model

y_pred_svm = svm_model.predict(X_test_scaled)             # Make predictions

print("SVM Classifier (Linear Kernel):")                  # Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.2f}")
print(f"F1 Score (macro): {f1_score(y_test, y_pred_svm, average='macro'):.2f}")

### DecisionTreeClassifier/Regressor 

In [ ]:
from sklearn.tree import DecisionTreeClassifier         # -->  classification, predicts a category or class label
from sklearn.metrics import accuracy_score, f1_score

# Create the classifier
clf_model = DecisionTreeClassifier(random_state=42)

# Fit the model on training data
clf_model.fit(X_train, y_train)

# Predict on test data
y_pred_clf = clf_model.predict(X_test)

# Evaluate
print("Decision Tree Classifier:")
print(f"Accuracy: {accuracy_score(y_test, y_pred_clf):.2f}")
print(f"F1 Score (macro): {f1_score(y_test, y_pred_clf, average='macro'):.2f}")

#----------------------------------------------------------------------------------------------------------------------------

from sklearn.tree import DecisionTreeRegressor         # -->  regression, predicts a numerical (continuous) value
from sklearn.metrics import mean_squared_error, r2_score

# Create the regressor
reg_model = DecisionTreeRegressor(random_state=42)

# Fit the model on training data
reg_model.fit(X_train, y_train)

# Predict on test data
y_pred_reg = reg_model.predict(X_test)

# Evaluate
print("Decision Tree Regressor:")
print(f"RMSE: {mean_squared_error(y_test, y_pred_reg, squared=False):.2f}")
print(f"R² Score: {r2_score(y_test, y_pred_reg):.2f}")

### GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Define PCA
pca = PCA(random_state = 42, n_components = clf[0].n_components_)

# Define GBC model
gbrt_clf = GradientBoostingClassifier(max_depth = 2,
                                      n_estimators = 6,
                                      learning_rate = 1.0,
                                      random_state = 42)

# Transform input data
X_train_transformed = pca.fit_transform(X_train)

# Fit model
gbrt_clf.fit(X_train_transformed, y_train)

# Predict
y_pred = gbrt_clf.predict(pca.transform(X_test))

print("PCA + Gradient Boosting Classifier:")
print(f"Accuracy score: {accuracy_score(y_test,y_pred):.2f}")
print(f"F1: {f1_score(y_test,y_pred, average= 'macro'):.2f}")

### RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# Define pipeline: (scaling is optional for Random Forest, but included for consistency)
pipeline = Pipeline([
    ('scaler', StandardScaler()),                       # Optional for RF, but good in pipelines
    ('rf', RandomForestClassifier(n_estimators=100,     # Number of trees
                                  max_depth=10,         # Optional limit on tree depth
                                  random_state=42))     # For reproducibility
])

# Train the model
pipeline.fit(X_train, y_train)

# Predict on test set
y_pred = pipeline.predict(X_test)

# Evaluate
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"F1 Score (macro): {f1_score(y_test, y_pred, average='macro'):.2f}")

### Bagging classifier (ensemble method)

In [ ]:
# Create an SVM model with a linear kernel inside a pipeline with StandardScaler
from sklearn.svm import SVC                             # use any classifier from scikit-learn
from sklearn.ensemble import BaggingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

svm_pipeline = make_pipeline(StandardScaler(), SVC(kernel="linear", random_state=42))
# Create a Bagging Classifier with 10 estimators using the SVM pipeline
bag_clf = BaggingClassifier(
    estimator=svm_pipeline,  
    n_estimators=10,
    n_jobs=-1, 
    random_state=42
)
# Train the Bagging model
bag_clf.fit(X_train, y_train)

### AdaBoost classifier(ensemble method)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score

# Define a pipeline with optional scaling and AdaBoost
pipeline = Pipeline([
    ('scaler', StandardScaler()),                           # Optional: useful for models sensitive to feature scale
    ('ada', AdaBoostClassifier(
        base_estimator=DecisionTreeClassifier(max_depth=1), # Weak learner (decision stump)
        n_estimators=100,                                   # Number of boosting rounds
        learning_rate=0.25,                                 # Step size for each estimator
        random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print(f"F1 Score (macro): {f1_score(y_test, y_pred, average='macro'):.2f}")


### SelfTrainingClassifier (semi-supervised)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import SelfTrainingClassifier

# Suppose you have some labels and some unknown (-1 indicates unknown label)
y_train_semi = y_train.copy()
y_train_semi[100:] = -1   # the first 100 rows have labels, the rest are unlabeled


# Define the base classifier
base_clf = RandomForestClassifier(random_state=42)

# Wrap it with SelfTrainingClassifier for semi-supervised learning
semi_supervised_clf = SelfTrainingClassifier(
    base_estimator=base_clf,
    criterion="threshold",
    threshold=0.99
)

# Fit on labeled + unlabeled data
# y_train_semi must have -1 for unlabeled instances
semi_supervised_clf.fit(X_train, y_train_semi)

# Make predictions
y_pred = semi_supervised_clf.predict(X_test)

### Grid Search(hyperparameter tuning technique)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Train the model here
clf

# Parameters for Grid Search
params = {'max_depth': [], 'max_features' : []}

# Perform Grid Search and train the model
clf2 = GridSearchCV(estimator = clf, param_grid = params)
clf2.fit(X_train, y_train)

# Predict 
y_pred = clf2.best_estimator_.predict(X_test)

best_params = clf2.best_estimator_.get_params()

print("Decision Tree Classifier:")
print(f"Best Max Depth: {best_params['max_depth']}")
print(f"Best Max Features: {best_params['max_features']}")
print(f"Accuracy score: {accuracy_score(y_test,y_pred):.2f}")
print(f"F1: {f1_score(y_test,y_pred, average= 'macro'):.2f}")
print()

for i_conf, conf in enumerate(clf2.cv_results_['params']):
    print(f"Configuration: {conf}  Mean Test Score: {clf2.cv_results_['mean_test_score'][i_conf]:.2f}")

### Grid Search with cross validation and pipeline

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# 1. Define pipeline: scaler + model
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

# 2. Define parameter grid for GridSearchCV (use step names!)
param_grid = {
    'svc__C': [0.1, 1, 10],
    'svc__kernel': ['linear', 'rbf']
}

# 3. Grid search with cross-validation
grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("Best params:", grid_search.best_params_)

### KMeans (clustering unsupervised learning algorithm)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# 1. Scale the input features (important for clustering)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Create the KMeans model
kmeans_model = KMeans(n_clusters=3, random_state=42, n_init='auto')  # You can change n_clusters as needed

# 3. Fit the model and predict cluster labels
y_kmeans = kmeans_model.fit_predict(X_scaled)

# 4. Evaluate clustering performance
silhouette = silhouette_score(X_scaled, y_kmeans)
print("K-Means Clustering:")
print(f"Number of clusters: {kmeans_model.n_clusters}")
print(f"Silhouette Score: {silhouette:.2f}")


### Cross Validation

In [ ]:
#A pipeline should be built, instead of using the scaled data as input, in order to avoid information "leakage" during
#formation of the different folds. A pipeline can be formed using either the "Pipeline" method, where naming of each 
#stage is performed by the user or the "make_pipeline" method which automatically names the pipeline stages.

from sklearn.model_selection import KFold, cross_val_score 
from sklearn.pipeline import Pipeline

lin_reg = Pipeline([
    ("scaler", StandardScaler()),
    ("regressor", LinearRegression())
    ])

k_folds = KFold(n_splits = 10)
scores = cross_val_score(lin_reg, x_train, y_train, cv = k_folds) 

print(f"Cross Validation Scores: {[np.round(s, 4) for s in scores]}")
print(f"Average CV Score: {scores.mean():.4f}, with STD: {scores.std():.4f}")
print(f"Number of CV Scores used in Average: {len(scores)}")

### Classification_report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_dtr))  

### Confusion_matrix

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_train, y_train_pred)            #Calculate the confusion matrix for the training set
cm

### F1-score

In [ ]:
# Compute and display the F1-score for classifier clf
#To calculate F1-score, you need to have y_pred, which means you must have used predict().
from sklearn.metrics import f1_score

y_pred = clf.predict(X_scaled)
f1 = f1_score(y, y_pred)
print(f"F1-score: {f1:.4f}")

### Αccuracy-score

In [ ]:
# Compute and display the accuracy-score for classifier clf
#To calculate accuracy, you need to have y_pred, which means you must have used predict().
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_scaled)
accuracy=accuracy_score(y_test,y_pred)
print(f"Accuracy with respect to test data:{accuracy:.3f}")

## Tensorflow.Keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [ ]:
# set the random seeds for TensorFlow, Python (random.seed()), and NumPy (np.random.seed())
tf.keras.utils.set_random_seed(42)

In [ ]:
# Load the Fashion MNIST dataset using Keras’ dataset module
 fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

### Neural Network

In [ ]:
# Set seeds for reproducibility
tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

# Model creation function
def create_model():
    model = Sequential([
        Input(shape=(14, 28)),
        Flatten(),
        Dense(128, activation='relu', kernel_initializer='he_normal'),
        Dense(64, activation='relu', kernel_initializer='he_normal'),
        Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

model = create_model()
initial_weights = model.get_weights()  # Store the random initial weights
model.fit(X_train, y_train, epochs=10)
model.summary()

In [ ]:
# Train the model and store history
history = model.fit(
    X_train_upper, y_train,
    validation_data=(X_valid_upper, y_valid),
    epochs=15,
    batch_size=32,
    verbose=1
)

### EarlyStopping

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Apply Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True  # Restore best weights after stopping
)
# Train with Early Stopping
 history_early = model.fit(
    X_train_upper, y_train,
    validation_data=(X_valid_upper, y_valid),
    epochs=15,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
 )

###  BatchNormalization

In [ ]:
model = Sequential([Input(shape=[14, 28]),
                    Flatten(),
                    Dense(128, use_bias=False),
                    BatchNormalization(),
                    Activation("relu"),
                    Dense( 64, use_bias=False),
                    BatchNormalization(),
                    Activation("relu"),
                    Dense( 32, use_bias=False),
                    BatchNormalization(),
                    Activation("relu"),
                    Dense(10, activation="softmax")])

model.summary()

### Dropout Regularization

In [ ]:
model = Sequential([Input(shape=[14, 28]),
                    Flatten(),
                    Dropout(rate=0.50),
                    Dense(128, activation = "relu"),
                    Dropout(rate=0.50),
                    Dense( 64, activation = "relu"),
                    Dropout(rate=0.50),
                    Dense( 32, activation = "relu"),
                    Dropout(rate=0.50),
                    Dense( 10, activation = "softmax")])

model.summary()

### Convolutional Neural Network (CNN)

In [ ]:
#  Build the Convolutional Neural Network (CNN) using TensorFlow's Keras API
model = tf.keras.models.Sequential([   # Creates a linear stack of layers, each layer feeds directly into the next
    # Feature extractor
    tf.keras.Input(shape=(28, 28, 1)), # Input images: 28×28 pixels, 1 channel (grayscale)
    layers.Conv2D(8, (5, 5), strides=(2, 2), padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(16, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.Flatten(), # Converts the 3D output of convolutions into a 1D vector for the classifier
    # Classifier
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(num_classes, activation='softmax') # softmax turns outputs into probabilities summing to 1
])

# Compile the model
 model.compile(optimizer='adam',                    # Adam Optimizer
              loss='categorical_crossentropy',      # categorical_crossentropy for one-hot encoded labels
              metrics=['accuracy'])   
 #  Model summary
 model.summary()

### Time Series

In [ ]:
# plot
sns.set(style="whitegrid") 

plt.figure(figsize=(12, 6))
sns.lineplot(data=df, x='', y='', label='', color='blue')

plt.xlabel('Date')
plt.ylabel('')
plt.title('')

plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Parameters
look_back = 30
forecast_horizon = 1  # Predict one step ahead

# Reshape the data into a column vector
sunspots_array = sunspots_array.reshape(-1, 1)

# Split the data into train/test sets (90% train)
train_size = int(len(sunspots_array) * 0.9)
train_data = sunspots_array[:train_size]
test_data = sunspots_array[train_size:]

# Normalize separately to avoid data leakage
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)

# Flatten the arrays for compatibility with sequence preparation
train_data_scaled = train_data_scaled.flatten()
test_data_scaled = test_data_scaled.flatten()

# Sequence-to-vector function
def create_seq2vec_dataset(data, look_back, forecast_horizon):
    """
    Converts a time series into supervised learning format:
    One input sample of `look_back` time steps predicts `forecast_horizon` steps ahead.
    """
    dataX, dataY = [], []
    for i in range(len(data) - look_back - forecast_horizon + 1):
        input_seq = data[i:i + look_back]
        output_seq = data[i + look_back:i + look_back + forecast_horizon]
        dataX.append(input_seq)
        dataY.append(output_seq)
    return np.array(dataX), np.array(dataY)

# Example usage:
X_train, y_train = create_seq2vec_dataset(train_data_scaled, look_back, forecast_horizon)
X_test, y_test = create_seq2vec_dataset(test_data_scaled, look_back, forecast_horizon)


###  Multilayer Percepton Model

In [ ]:
from keras.models import Sequential
from keras.layers import Input, BatchNormalization, Dense
from keras.optimizers import AdamW
from keras.callbacks import EarlyStopping

output_size= y_target.shape[1] 

# Define model
model = Sequential([                                                  # Initializes a Sequential model
    Input(shape=(look_back,)),                                        # Input layer with shape = look_back
    BatchNormalization(),                                             # Batch Norm layer
    Dense(50, activation='selu', kernel_initializer='lecun_normal'),  # Dense(50)
    Dense(25, activation='selu', kernel_initializer='lecun_normal'),  # Dense(25)
    Dense(output_size)                                                # Οutput layer
])                                              

# Compile the model
model.compile(
    optimizer=AdamW(learning_rate=0.001),                            # Optimizer with decoupled weight decay
    loss='mse',                                                      # Mean Squared Error for regression
    metrics=['mae']                                                  # Mean Absolute Error as a metric
)                                                                    

# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',                                              # Monitor validation loss
    patience=2,
    restore_best_weights=True
)

# Fit the model
history = model.fit(
    X_input,
    y_target,
    epochs=30,
    batch_size=32,
    validation_split=0.1,                                            # Use 10% of data for validation
    callbacks=[early_stop],
    verbose=1
)

# Model summary
print("\nModel Summary:")
model.summary()

### Stacked Autoencoder (for MNIST)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define input shape and parameters
input_shape = (28, 28)                                # Input is a 28x28 image (e.g. from MNIST)
flatten_shape = 28 * 28                               # Flattened shape = 784
latent_dim = 50                                       # Size of the encoded (compressed) representation

# Function to build the autoencoder model
def build_autoencoder():
    # Encoder network
    encoder = models.Sequential([
        tf.keras.Input(shape=input_shape),            # Input layer (28x28 image)
        layers.Flatten(),                             # Flatten image to vector of size 784
        layers.Dense(256, activation='relu'),         # Dense hidden layer
        layers.Dense(latent_dim, activation='relu', name='latent')  # Encoded representation (latent space)
    ])

    # Decoder network
    decoder = models.Sequential([
        tf.keras.Input(shape=(latent_dim,)),          # Input is latent vector (size 50)
        layers.Dense(256, activation='relu'),         # Hidden layer
        layers.Dense(flatten_shape, activation='sigmoid'),  # Output layer with sigmoid activation (range 0–1)
        layers.Reshape(input_shape)                   # Reshape back to 28x28 image
    ])

    # Combine encoder and decoder into a single model
    autoencoder = models.Sequential([encoder, decoder])

    # Compile the model
    autoencoder.compile(
        optimizer=tf.keras.optimizers.Nadam(learning_rate=1e-4),  # Nadam optimizer with small learning rate
        loss='binary_crossentropy'                                # Binary cross-entropy loss for pixel-wise comparison
    )
    return autoencoder

# Build and summarize the model
autoencoder = build_autoencoder()
autoencoder.summary()

# Save the autoencoder weights
autoencoder_weights = autoencoder.get_weights()  # List of all trainable weights in the model

### GAN

In [ ]:
# Define alias for Dense layer
Dense = tf.keras.layers.Dense

# Size of the noise vector (latent space)
codings_size = 30

# Batch size for training
batch_size = 32

# Define the Generator model
generator = tf.keras.Sequential([
    tf.keras.layers.InputLayer(shape=(codings_size,)),               # Input: random noise vector of size 30
    Dense(100, activation="relu", kernel_initializer="he_normal"),   # Dense layer with 100 units and ReLU activation
    Dense(150, activation="relu", kernel_initializer="he_normal"),   # Dense layer with 150 units and ReLU activation
    Dense(28 * 28, activation="tanh"),                                # Output layer to produce 784 values in [-1, 1]
    tf.keras.layers.Reshape([28, 28])                                 # Reshape to image format (28x28)
])
generator.summary()

# Define the Discriminator model
discriminator = Sequential([
    InputLayer(shape=(28, 28)),                                       # Input: 28x28 image
    Flatten(),                                                        # Flatten image to vector
    Dense(150, activation="relu", kernel_initializer="he_normal"),    # Dense layer with 150 units
    Dense(100, activation="relu", kernel_initializer="he_normal"),    # Dense layer with 100 units
    Dense(1, activation="sigmoid")                                    # Output layer: probability that input is real
])
discriminator.summary()

# Compile the Discriminator (standalone) with training enabled
discriminator.trainable = True
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")

# Freeze Discriminator's weights when training the combined GAN
discriminator.trainable = False

# Define the combined GAN model: Generator followed by Discriminator
gan = tf.keras.Sequential([generator, discriminator])

# Compile the full GAN model
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

# Create a tf.data.Dataset from X_train and prepare it for training
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)       # Shuffle data
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)      # Batch and prefetch for performance


In [ ]:
def train_gan(gan, dataset, batch_size, codings_size, n_epochs):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print(f"Epoch {epoch + 1}/{n_epochs}")
        for X_batch in dataset:
            #Enable discriminator training
            discriminator.trainable = True
            # Phase 1 train discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.concat([
                tf.zeros((batch_size, 1)),
                tf.ones((batch_size, 1)) * 0.9
            ], axis=0)
            discriminator.train_on_batch(X_fake_and_real, y1)
            #Freeze discriminator for GAN training (generator only)
            discriminator.trainable = False
            # Phase 2 train generator via GAN
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.ones((batch_size, 1))  # trick discriminator: all real for generator training
            gan.train_on_batch(noise, y2)
        # Plot generated images after each epoch
        plot_multiple_images(generated_images.numpy(), 32)
        
train_gan(gan, dataset, batch_size, codings_size, n_epochs=10)